In [1]:
import pandas as pd
from src import utils
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import q_stat
from statsmodels.tsa.stattools import adfuller

In [ ]:
df_features_indicators_not_null = None

In [ ]:
X = df_features_indicators_not_null.drop(columns = ["last_close"])
y = df_features_indicators_not_null.last_close

In [ ]:
filter_test = X.index.max() - pd.Timedelta(days = 4)


In [ ]:
train = y.loc[y.index < filter_test]
test = y.loc[y.index > filter_test]

In [ ]:

days30 = train.index.max() - pd.Timedelta(days = 30)
days60 = train.index.max() - pd.Timedelta(days = 60)
days90 = train.index.max() - pd.Timedelta(days = 90)

In [ ]:
train_30 = train.loc[train.index > days30]
train_60 = train.loc[train.index > days60]
train_90 = train.loc[train.index > days90]

In [ ]:
y.plot();

In [ ]:
train_30.plot(title = '30 days');

In [ ]:
train_60.plot(title = '60 days');

In [ ]:
train_90.plot(title = '90 days');

In [ ]:

info = dict()

for i in range(1,int(len(train_60)*1/4)+1):
  info.update({f'Lag {i}': train_60.autocorr(i)})
  

correlations = pd.DataFrame(pd.Series(info)).rename(columns = {0:'person_correlation'})

correlations
     

In [ ]:

q_statistic, p_values = q_stat(correlations['person_correlation'], len(train_60))
correlations['q_statistic'] = q_statistic.round(2)
correlations['prob_q'] = p_values.round(2)
correlations['random'] = correlations['prob_q'].apply(lambda p: utils.qstat_is_random(p))

In [ ]:
correlations

In [ ]:
class StationarityTests:
    def __init__(self, significance=.05):
        self.SignificanceLevel = significance
        self.pValue = None
        self.isStationary = None
    def ADF_Stationarity_Test(self, timeseries, printResults = True):
        #Dickey-Fuller test:
        adfTest = adfuller(timeseries, autolag='AIC')
        
        self.pValue = adfTest[1]
        
        if (self.pValue<self.SignificanceLevel):
            self.isStationary = True
        else:
            self.isStationary = False
        
        if printResults:
            dfResults = pd.Series(adfTest[0:4], index=['ADF Test Statistic','P-Value','# Lags Used','# Observations Used'])
            #Add Critical Values
            for key,value in adfTest[4].items():
                dfResults['Critical Value (%s)'%key] = value
            print('Augmented Dickey-Fuller Test Results:')
            print(dfResults)
            print("That time series is stationary? {0}".format(self.isStationary))

In [ ]:
sTest = StationarityTests()
sTest.ADF_Stationarity_Test(train_60, printResults = True)


In [ ]:
train_60_diff = train_60.diff().dropna()

In [ ]:
train_60_diff.plot()
train_60_diff.rolling(window = 5).mean().plot(label = '5 days rolling mean',style= '--')
train_60_diff.rolling(window = 3).mean().plot(label = '3 days rolling mean',style='--');    
plt.legend();

In [ ]:
plot_acf(train_60.diff().dropna(), lags = 30, title = 'ACF 60 days');

In [ ]:
plot_pacf(train_60.diff().dropna(), lags = 29, title = 'PACF 60 days');

In [ ]:
y